In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RAG - Developer Code Chat

# Text Classification with Generative Models on Vertex AI


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/examples/developer_code_chat.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fexamples%2Fdeveloper_code_chat.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/examples/developer_code_chat.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/use-cases/retrieval-augmented-generation/developer_code_chat.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Charu Shelar](https://github.com/CharulataShelar) |
|Reviewer(s) | [Erwin Huizenga](https://github.com/erwinh85) |

## Overview

This notebook showcases the development of an AI-powered learning assistant. This assistant is designed to help users, such as programmers or students, learn more about programming languages. The assistant answers users' questions using internal documents. It can assist end users with coding tasks, answer questions, and generate code. The solution has been built using the custom RAG approach and Gemini model (Gemini Pro 1.0). It stores the responses in BigQuery. This allows for the caching of more common queries and analytics.


## As a developer, you will learn the steps to implement the complete solution, i.e. :

1. To embed the document and create a vector search index using Vector Search (previously known as Matching Engine).
Upload new document in GCS bucket
Separate tab on the UI to allow end users to index newly added documents.
Python code file used here: learning_assistant/generate_embeddings_main.py

2. To build RAG (Retrieval-Augmented Generation) for intra document search
Use Gemini model to allow chat like conversation and retain the session conversation history limited to last N messages (3 previous messages in this case )
Answer programming questions using indexed documents.
Answer coding questions using the Gemini model if knowledge base does not have the relevant context/content
To prevent hallucinations and maintain appropriate responses, the solution demonstrates how to guardrail the system's response to predetermined programming languages when handling user queries. List of supported programming languages can be configured in config.ini file
Python code file used here: learning_assistant/learning_assistant.py

4. To build chat UI interface using Gradio
5. Integrate BigQuery to save responses
Save the responses generated by the chatbot agent in Bigquery table for caching and further analytics by session ID: genai-github-assets.genai_data.assistant_feedback
(Note: New session ID is created for each new gradio instance)


## GCP services used:

1. Vector Search (previously Matching Engine)
2. GenAI Model - Gemini Pro 1.0, textembedding-gecko
3. BigQuery


## Solution Design Flow

![genAI Asset Learning assistent](https://screenshot.googleplex.com/A7voRwxx6RRm9sV.png)

### Data Ingestion Stages:
1. Developer team having access to new learning content can upload it to the GCS bucket.

2. Data Ingestion Pipeline will fetch documents from the GCS bucket (here "gs://genai-prod-vme-embedding/references") and create chunks based on the document sizes.

3. Further Data Ingestion Pipeline will get the embeddings for each page using Vertex AI Embeddings model API and index to Vector Search.


### Response Generation Stages:
1. The user starts a natural language query through a Gradio Chat User Interface (UI).

2. Intent classification is done using Gemini model. It classifies the message into one of the following intents: WELCOME, PROGRAMMING_QUESTION_AND_ANSWER, WRITE_CODE, FOLLOWUP, or CLOSE.

3. For the WRITE_CODE intent, the Gemini model is used to generate code using its coding capability.

4. For the PROGRAMMING_QUESTION_AND_ANSWER intent, custom orchestration (RAG) retrieves context relevant to the user query from Vector Search and summarises relavent contexts. If the answer is not found, the user query is routed to the Gemini Model to respond using its knowledge.

5. For the FOLLOWUP intent, such as explaining more or writing code for previous responses, the Gemini Model is used to generate responses using its code capability.

6. For the WELCOME and CLOSE intents, the Gemini model is used to generate appropriate responses.


## Getting Started

### Install Vertex AI SDK and other required packages


In [ ]:
# !pip3 install --upgrade --user --quiet google-cloud-aiplatform
# !pip install gradio
# !pip install langchain
# !pip install pypdf
# !pip install -U langchain-google-vertexai

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
# import sys

# if "google.colab" in sys.modules:
#     import IPython

#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Clone the source code from CSR repo

In [ ]:
import os

if os.path.isdir("rag_developer_code_chat"):
    # If the folder already exists:
    print("rag_developer_code_chat folder already exists")

    # 1. Either pull the latest repo code in the folder before running the notebook
    # !git reset --hard
    # !git pull

    # 2. or remove entire folder and clone the repo again
    !rm -r rag_developer_code_chat -f
    !gcloud source repos clone rag_developer_code_chat --project=genai-github-assets
else:
    !gcloud source repos clone rag_developer_code_chat --project=genai-github-assets

rag_developer_code_chat folder already exists
Cloning into '/content/rag_developer_code_chat'...
remote: Finding sources: 100% (5/5)
remote: Total 65 (delta 31), reused 62 (delta 31)
Receiving objects: 100% (65/65), 66.94 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (31/31), done.
Project [genai-github-assets] repository [rag_developer_code_chat] was cloned to [/content/rag_developer_code_chat].


### Change the working directory to cloned folder

In [ ]:
import os

print("Default working directory : ", os.getcwd())

path = "rag_developer_code_chat"
os.chdir(path)
print("Working directory changed to : ", os.getcwd())

Default working directory :  /content
Working directory changed to :  /content/rag_developer_code_chat


### Import required packages

In [ ]:
import configparser
import logging
import uuid

import gradio as gr
from learning_assistant.generate_embeddings_main import index_documents
from learning_assistant.learning_assistant import get_bucket_content
from learning_assistant.utils.intent_routing import IntentRouting
from learning_assistant.utils.log_response_bq import LogDetailsInBQ
from learning_assistant.utils.qna_vector_search import QnAVectorSearch
from learning_assistant.utils.vertex_matching_engine_utils import \
    deploy_index_endpoint
from vertexai.generative_models import GenerativeModel

### Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
PROJECT_ID = "genai-github-assets"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Chat interface using gradio app

#### Mount Learning Assistant app with gradio

In [ ]:
def create_app() -> gr.Blocks:
    """
    Initializes and configures the Gradio web interface for the Learning Assistant application.

    Key Functions:
        * Reads app configuration from 'learning_assistant/config.ini'.
        * Sets up logging.
        * Deploys the index endpoint (if necessary).
        * Instantiates core components: IntentRouting, QnAVectorSearch, LogDetailsInBQ.
        * Loads language models for intent classification and chat.
        * Defines Gradio interface elements, including chatbot and feedback logging.
        * Handles user input and orchestrates responses.

    Returns:
        gr.Blocks: The configured Gradio interface object.
    """

    # Settings
    config_file = "learning_assistant/config.ini"
    config = configparser.ConfigParser()
    config.read(config_file)

    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    deploy_index_endpoint(logger)

    genai_assistant = IntentRouting(config_file=config_file, logger=logger)
    genai_qna = QnAVectorSearch(config_file=config_file, logger=logger)
    bq_logger = LogDetailsInBQ(config_file=config_file)

    model = GenerativeModel(config["model_parameter_classify"]["classify_model_name"])

    chat_model = GenerativeModel(
        config["model_parameter_classify"]["classify_model_name"]
    )
    chat = chat_model.start_chat(history=[])
    _ = chat.send_message(
        """You are Programming Language Learning Assistant.
            Your task is to undersand the question and answer descriptive answer for the same.

            Instructions:
            1. If programming language is not mentioned, then use java as default programming language to write a code.
            2. Strictly follow the instructions mentioned in the question.
            3. If the question is not clear then you can answer "I apologize, but I am not able to understand the question. Please try to elaborate and rephrase your question."

            If the question is about other programming language then DO NOT provide any answer, just say "I apologize, but I am not able to understand the question. Please try to elaborate and rephrase your question."
    """
    )

    css_code = """
    .form {border-color: #ecedfc !important}
    .user {border-color: #f9fafb !important;background-color: #fbfbff !important}
    .bot {border-color: #f9fafb !important;background-color: #f7f0ff !important}
    """

    with gr.Blocks(css=css_code) as demo:
        with gr.Tab("Learning Assistant"):
            bot_message = "Hi there! I'm Generative AI powered Learning Assistant. I can help you with coding tasks, answer questions, and generate code. Just ask me anything you need, and I'll do my best to help!"  # pylint: disable=C0301:line-too-long

            session_state = str(uuid.uuid4())  # "test"
            logger.info("session_state : %s", session_state)

            chatbot = gr.Chatbot(
                height=600,
                label="",
                value=[[None, bot_message]],
                avatar_images=(
                    None,
                    "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/smart_assistant/default/24px.svg",
                ),  # # pylint: disable=C0301:line-too-long
                elem_classes="message",
                show_label=False,
            )
            msg = gr.Textbox(
                scale=4,
                label="",
                placeholder="Enter your question here..",
                elem_classes=["form", "message-row"],
            )

            def respond(
                message, chat_history
            ):  # pylint: disable=W0621:redefined-outer-name
                """Sending response to gradio"""
                # intent
                (response, intent, answer_reference,) = genai_assistant.classify_intent(
                    message,
                    session_state,
                    model,
                    chat_model,
                    chat_history,
                    genai_qna,
                )

                # append response to history
                chat_history.append((message, response + answer_reference))

                bq_logger.save_response(
                    question=message,
                    intent=intent,
                    response=response + answer_reference,
                    session_state=session_state,
                )

                return "", chat_history

            msg.submit(respond, [msg, chatbot], [msg, chatbot])
        with gr.Tab("Data"):
            data_df = get_bucket_content()
            # data
            _ = gr.Dataframe(data_df)
            progress = gr.Textbox(label="Index Document Status")
            btn = gr.Button(value="Index Documents")
            btn.click(index_documents, outputs=[progress])

    return demo

#### Launch the gradio app to view the chatbot

In [ ]:
demo = create_app()
demo.launch(width="80%", share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a77de6cc4e8869f051.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        SYSTEM: You are genai Programming Language Learning Assistant helping the students answer their questions based on following context. Explain the answers in detail for students.

        Instructions:
        1. Think step-by-step and then answer.
        2. Explain the answer in detail.
        3. If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
        4. If the context is empty, just say "I could not find any references that are directly related to your question."

        Context:
        

        What is the Detailed explanation of answer of following question?
        Question: can you help identify difference between list and set?
        Detailed explanation of Answer:

> Finished chain.

> Finished chain.

> Finished chain.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        SYSTEM: You are genai Programming Language Learning Assistant helping the students answer their questions based on following context. Explain the answers in detail for students.

        Instructions:
        1. Think step-by-step and then answer.
        2. Explain the answer in detail.
        3. If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
        4. If the context is empty, just say "I could not find any references that are directly related to your question."

        Context:
        

        What is the Detailed explanation of answer of following question?
        Question: examples of list
        Detailed explanation of Answer:

> Finished chain.

> Finished chain.

> Finished chain.
